In [4]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
M_sun = const.M_sun.value

In [ ]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.sqrt(np.dot(r_vec, r_vec))
    return r

def findCM(x_1, x_2, m_1,m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

In [ ]:
# Rescaling to dimensionless

def findR_S(m_1, m_2):
    c = const.c.value
    G = const.G.value
    R_S = 2 * (m_1 + m_2) * G/c**2
    return R_S

def SItoDL(s, t_max, dt):
    c = const.c.value
    G = const.G.value

    x_1, x_2, v_1, v_2, m_1, m_2 = s
    R_S = findR_S(m_1, m_2)

    X_1, X_2 = x_1/R_S, x_2/R_S
    V_1, V_2 = v_1/c, v_2/c
    M_1, M_2 = m_1 * G/(c**2*R_S), m_2 * G/(c**2*R_S)

    S = X_1, X_2, V_1, V_2, M_1, M_2
    T_max, dT = t_max * c/R_S, dt * c/R_S

    return S, T_max, dT, R_S



In [1]:
def accelerationN(x_1, x_2, m_1, m_2, DL):
    if DL == True:
        G = 1
    else:
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    n = r_vec/r

    a_1 = - G*m_2*n/r**2
    a_2 =   G*m_1*n/r**2
    return np.array([a_1, a_2])


def accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2, DL, α):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    v_vec = v_1 - v_2
    n = r_vec/r

    a_1 = - G*m_2/r**2*n
    + α* 1/c**2*((5*G**2*m_1*m_2/r**3 + 4*G**2*m_2**2/r**3 + G*m_2/r**2*(3/2*np.dot(n, v_2)**2 - np.dot(v_1, v_1) + 4*np.dot(v_1, v_2) - 2*np.dot(v_2, v_2)))*n
    + G*m_2/r**2 *(4*np.dot(n, v_1) - 3*np.dot(n, v_2))*v_vec)

    a_2 = - G*m_1/r**2*(-n)
    + α * 1/c**2*((5*G**2*m_2*m_1/r**3 + 4*G**2*m_1**2/r**3 + G*m_1/r**2*(3/2*np.dot((-n), v_1)**2 - np.dot(v_2, v_2) + 4*np.dot(v_2, v_1) - 2*np.dot(v_1, v_1)))*(-n)
    + G*m_1/r**2 *(4*np.dot((-n), v_2) - 3*np.dot((-n), v_1))*(-v_vec))
    return np.array([a_1, a_2])


def accelerationGR(x_1, x_2, v_1, v_2, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    v_vec = v_1 - v_2

    l = np.cross(r_vec, v_vec)

    a_1 = - G*m_2/r**3*(1 + 3*l**2/(c**2 * r**2))*r_vec
    a_2 =   G*m_1/r**3*(1 + 3*l**2/(c**2 * r**2))*r_vec
    return np.array([a_1, a_2])

def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

In [5]:
def run_model(s, t_max, dt, EIH=True, GR=False, DL=False, α = 1):
    if DL == True:
        s, t_max, dt, R_S = SItoDL(s, t_max, dt)
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s

    pos1 = np.zeros((2, int(t_max/dt)))
    pos2 = np.zeros((2, int(t_max/dt)))
    pos_CM = np.zeros((2, int(t_max/dt)))

    for i in range(int(t_max/dt)):
        pos1[:,i] = x_1
        pos2[:,i] = x_2
        pos_CM[:,i] = findCM(x_1, x_2, m_1, m_2)
        if EIH == True:
            a_1, a_2 = accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2, DL, α)
        if GR == True:
            a_1, a_2 = accelerationGR(x_1, x_2, v_1, v_2, m_1, m_2, DL)
        else:
            a_1, a_2 = accelerationN(x_1, x_2, m_1, m_2, DL)
        v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)
    return pos1, pos2, pos_CM

In [1]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def orbPlotter(pos1, pos2, posCM, aspect = 1, filename='', CM = False, save = False, show = True, DL = False, figsize=(7,7)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b--', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'bo', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'b.', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r--', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'ro', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'r.', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./PLOTS/{filename}.png', dpi=300, transparent = False)
    return

def orbPlotter_2(pos1, pos2, posCM, xlim, ylim, aspect = 1, filename='', CM = False, save = False, show = True, DL = False, figsize=(7,7)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    # xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b--', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'bo', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'b.', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r--', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'ro', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'r.', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./PLOTS/{filename}.png', dpi=300, transparent = False)
    return

def prettyOrbPlotter(pos1, pos2, posCM, aspect = 1, filename='', CM = False, save = False, show = True):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = 0


    fig, ax = plt.subplots(figsize=(11,11))
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    # ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    # ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b--', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r--', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    # ax.set_xlabel('$R_S$', fontsize = 15)
    # ax.set_ylabel('$R_S$', fontsize = 15)
    # ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    # ax.set_title(f'{filename}', fontsize = 20)
    # ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
                
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Two_Body/Pretty/{filename}_pretty.png', dpi=300, transparent = False)
    return